# Requirements Traceability Analysis and Model Comparison
**Analysis and visualization of requirements traceability model performance with cross-project comparison, LLM vs baseline evaluation, confusion matrix analysis, and role-based performance assessment using Neo4j data integration.**

In [ ]:
# Cell [0] - Setup and Imports
# Purpose: Import all required libraries and configure environment settings
# Dependencies: pandas, numpy, neo4j, matplotlib, seaborn, dotenv, re
# Breadcrumbs: Setup -> Imports

import os
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
from datetime import datetime
from dotenv import load_dotenv
from neo4j import GraphDatabase
from matplotlib.patches import Patch
from matplotlib.colors import LinearSegmentedColormap

def setup_environment():
    """
    Configure logging and load environment variables for traceability analysis
    
    Returns:
        tuple: (config_dict, logger_instance) containing configuration parameters and logger
    """
    # Configure logging
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    
    # Load environment variables
    load_dotenv()
    
    # Neo4j credentials from environment variables
    config = {
        'NEO4J_URI': os.getenv('NEO4J_URI'),
        'NEO4J_USER': os.getenv('NEO4J_USER'),
        'NEO4J_PASSWORD': os.getenv('NEO4J_PASSWORD')
    }
    
    # Set figure style for consistent visualization
    plt.style.use('seaborn-v0_8-whitegrid')
    sns.set_context("talk")
    
    logger.info("Environment setup completed for traceability analysis")
    return config, logger

# Execute setup
CONFIG, logger = setup_environment()
NEO4J_URI = CONFIG['NEO4J_URI']
NEO4J_USER = CONFIG['NEO4J_USER']
NEO4J_PASSWORD = CONFIG['NEO4J_PASSWORD']

In [ ]:
# Cell [1] - Neo4j Connection Setup
# Purpose: Create and test connection to Neo4j database containing traceability data
# Dependencies: neo4j, logging
# Breadcrumbs: Setup -> Database Connection

def create_neo4j_driver(uri=None, user=None, password=None):
    """
    Create and return a Neo4j driver instance for database connectivity
    
    Parameters:
        uri (str, optional): Neo4j URI. Defaults to NEO4J_URI from environment.
        user (str, optional): Neo4j username. Defaults to NEO4J_USER from environment.
        password (str, optional): Neo4j password. Defaults to NEO4J_PASSWORD from environment.
    
    Returns:
        GraphDatabase.driver: Connected Neo4j driver instance
        
    Raises:
        Exception: If connection to Neo4j fails
    """
    try:
        # Use parameters if provided, otherwise use globals from setup_environment
        _uri = uri if uri is not None else NEO4J_URI
        _user = user if user is not None else NEO4J_USER
        _password = password if password is not None else NEO4J_PASSWORD
        
        driver = GraphDatabase.driver(_uri, auth=(_user, _password))
        logger.info("Successfully connected to Neo4j database")
        return driver
    except Exception as e:
        logger.error(f"Failed to connect to Neo4j: {str(e)}")
        raise

def test_neo4j_connection(driver):
    """
    Test the Neo4j database connection
    
    Parameters:
        driver: Neo4j driver instance to test
        
    Returns:
        bool: True if connection successful, False otherwise
    """
    try:
        with driver.session() as session:
            result = session.run("RETURN 1 as test").single()
            if result and result["test"] == 1:
                print("✓ Successfully connected to Neo4j database")
                return True
            else:
                print("✗ Failed to connect to Neo4j database")
                return False
    except Exception as e:
        logger.error(f"Database connection test failed: {str(e)}")
        print("✗ Failed to connect to Neo4j database")
        return False

# Create Neo4j driver
driver = create_neo4j_driver()

# Test connection
test_neo4j_connection(driver)

In [ ]:
# Cell [2] - Query Model Data from Neo4j
# Purpose: Retrieve individual model data points for visualization and analysis
# Dependencies: neo4j, pandas, json, logging
# Breadcrumbs: Data Acquisition -> Model Data Retrieval

def query_model_data(driver):
    """
    Query model data from Neo4j for all model types including metrics analysis
    
    Parameters:
        driver: Neo4j driver instance for database connection
        
    Returns:
        pd.DataFrame: DataFrame containing model data with parsed JSON metrics,
                     display names, and metadata. Empty DataFrame if no data found.
                     
    Raises:
        Exception: If database query fails or data parsing encounters errors
    """
    try:
        # Query for metrics analysis data including model_data
        query = """
        MATCH (p:Project)-[r:HAS_METRICS_ANALYSIS]->(m:MetricsAnalysis)
        WHERE m.analysis_type = 'whisker_chart'
        RETURN 
            p.name as project_name,
            m.model_type as model_type,
            m.model_data as model_data,
            m.model_count as model_count,
            m.created_at as created_at
        ORDER BY p.name, m.model_type
        """
        
        with driver.session() as session:
            results = session.run(query).data()
            
            if not results:
                logger.warning("No model data found in Neo4j")
                return pd.DataFrame()
                
            # Convert results to DataFrame
            model_df = pd.DataFrame(results)
            
            # Parse JSON data
            model_df['model_data_parsed'] = model_df['model_data'].apply(
                lambda x: json.loads(x) if isinstance(x, str) else x
            )
            
            # Make sentence transformer/tf-idf human readable
            model_df['display_model_type'] = model_df['model_type'].apply(
                lambda x: "Sentence Transformer/TF-IDF" if x == "sentence_transformer_tf_idf" else x
            )
                
            logger.info(f"Retrieved model data for {model_df['model_type'].nunique()} model types across {model_df['project_name'].nunique()} projects")
            
            return model_df
            
    except Exception as e:
        logger.error(f"Error querying model data: {str(e)}")
        return pd.DataFrame()

# Query model data for both sentence transformers and LLMs
model_df = query_model_data(driver)

# Display summary of retrieved data
if not model_df.empty:
    print(f"\nRetrieved model data for {model_df['project_name'].nunique()} projects:")
    print("=" * 80)
    
    for project in model_df['project_name'].unique():
        project_data = model_df[model_df['project_name'] == project]
        print(f"\nProject: {project}")
        print("-" * 40)
        
        for _, row in project_data.iterrows():
            print(f"  - {row['display_model_type']}: {row['model_count']} models analyzed on {row['created_at']}")
            
    # Display example of model data structure
    print("\nExample model data structure:")
    print("-" * 40)
    first_model_data = model_df.iloc[0]['model_data_parsed']
    if first_model_data:
        for metric, models in list(first_model_data.items())[:2]:  # Show first 2 metrics only
            print(f"  Metric: {metric}")
            for model_name, score in list(models.items())[:3]:  # Show first 3 models only
                print(f"    - {model_name}: {score}")
else:
    print("No model data found. Please run analysis notebooks first to generate data.")

In [ ]:
# Cell [3] - Process and Prepare Model Data for Visualization
# Purpose: Extract and organize individual model data points for comprehensive visualization
# Dependencies: pandas, logging
# Breadcrumbs: Data Processing -> Model Data Organization

def extract_raw_model_data(model_df):
    """
    Extract raw model data points from Neo4j data and organize by project, metric, and model
    
    Parameters:
        model_df (pd.DataFrame): DataFrame containing model data from Neo4j with parsed JSON
        
    Returns:
        dict: Nested dictionary structure organized as:
              {project: {metric: {model_type: {model_name: [scores]}}}}
              Returns empty dict if input DataFrame is empty or invalid.
              
    Notes:
        - Converts raw metric names to human-readable display names
        - Handles multiple score entries per model for statistical analysis
        - Supports metrics: accuracy, balanced_accuracy, precision, recall, f1_score, f2_score, matthews_corr
    """
    if model_df.empty:
        return {}
    
    # Initialize data structure
    # Format: {project: {metric: {model_type: {model_name: [scores]}}}}
    organized_data = {}
    
    # Define metrics to extract
    metrics_to_extract = [
        'accuracy', 'balanced_accuracy', 'precision', 'recall', 
        'f1_score', 'f2_score', 'matthews_corr'
    ]
    
    # Human readable metric names mapping
    metric_display_names = {
        'accuracy': 'Accuracy',
        'balanced_accuracy': 'Balanced Accuracy',
        'precision': 'Precision',
        'recall': 'Recall',
        'f1_score': 'F1 Score',
        'f2_score': 'F2 Score',
        'matthews_corr': 'Matthews Correlation'
    }
    
    for _, row in model_df.iterrows():
        project = row['project_name']
        model_type = row['display_model_type']
        model_data = row['model_data_parsed']
        
        if not model_data or not isinstance(model_data, dict):
            continue
            
        if project not in organized_data:
            organized_data[project] = {}
            
        # Process each metric in model_data
        for metric_key, models in model_data.items():
            # Get display name for metric
            metric_display = None
            for raw_name, display_name in metric_display_names.items():
                if raw_name.lower() == metric_key.lower() or raw_name.replace('_', '') == metric_key.replace('_', ''):
                    metric_display = display_name
                    break
                    
            if not metric_display:
                # If no match found, use capitalized version of the key
                metric_display = metric_key.replace('_', ' ').title()
                
            if metric_display not in organized_data[project]:
                organized_data[project][metric_display] = {}
                
            if model_type not in organized_data[project][metric_display]:
                organized_data[project][metric_display][model_type] = {}
                
            # Add model scores
            for model_name, score in models.items():
                if isinstance(score, (int, float)):
                    # Add to existing list or create new one
                    if model_name in organized_data[project][metric_display][model_type]:
                        organized_data[project][metric_display][model_type][model_name].append(score)
                    else:
                        organized_data[project][metric_display][model_type][model_name] = [score]
    
    return organized_data

# Extract raw model data
raw_model_data = extract_raw_model_data(model_df)

# Display a summary of the organized raw data
if raw_model_data:
    print("\nRaw model data summary:")
    print("=" * 80)
    
    for project, metrics in raw_model_data.items():
        print(f"\nProject: {project}")
        print("-" * 40)
        
        for metric, model_types in metrics.items():
            print(f"  Metric: {metric}")
            for model_type, models in model_types.items():
                total_points = sum(len(scores) for scores in models.values())
                print(f"    - {model_type}: {len(models)} models, {total_points} total data points")
                # Show example data
                if models:
                    first_model = next(iter(models.items()))
                    print(f"      Example: {first_model[0]}: {first_model[1][:5]}...")
else:
    print("No raw model data available for visualization.")

In [ ]:
# Cell [4] - Create Metric-Specific Visualizations with Individual Data Points
# Purpose: Generate comprehensive boxplots with individual data points for model type comparison
# Dependencies: pandas, matplotlib, seaborn, numpy
# Breadcrumbs: Visualization -> Metric Comparison Charts

def create_metric_visualizations_with_points(raw_model_data):
    """
    Create comprehensive visualizations for each metric in each project comparing model performance
    
    Parameters:
        raw_model_data (dict): Nested dictionary containing organized model data from extract_raw_model_data()
                              Structure: {project: {metric: {model_type: {model_name: [scores]}}}}
                              
    Returns:
        None: Displays visualizations directly using matplotlib/seaborn
        
    Notes:
        - Creates separate boxplots for each metric in each project
        - Includes individual data points with jitter for detailed analysis
        - Generates statistical heatmaps with min, max, mean, median, quartiles
        - Uses consistent color mapping across visualizations
        - Saves high-resolution PNG files for each visualization
        - Prioritizes Sentence Transformer/TF-IDF models in display order
    """
    # Import numpy for jitter function
    import numpy as np
    
    if not raw_model_data:
        print("No data available for visualization.")
        return
    
    # Create dynamic color map based on actual model types in the data
    all_model_types = set()
    for project, metrics in raw_model_data.items():
        for metric, model_types in metrics.items():
            all_model_types.update(model_types.keys())
    
    # Define color palette
    color_palette = ['gold', 'skyblue', 'lightgreen', 'lightpink', 'mediumpurple', 'lightcoral', 'lightblue', 'lightsalmon']
    
    # Create color map dynamically
    color_map = {}
    for i, model_type in enumerate(all_model_types):
        color_map[model_type] = color_palette[i % len(color_palette)]
    
    # Process each project
    for project, metrics in raw_model_data.items():
        print(f"\nGenerating visualizations for project: {project}")
        print("-" * 80)
        
        # Create a figure for each metric
        for metric_name, model_types in metrics.items():
            # Skip if no model types
            if not model_types:
                print(f"  Skipping metric {metric_name}: No model types")
                continue
                
            # Print status update
            print(f"  Processing metric: {metric_name}")
            
            # Create combined dataset for all model types
            all_models_data = []
            for model_type, models in model_types.items():
                for model_name, scores in models.items():
                    # Add each score as a separate row
                    for score in scores:
                        all_models_data.append({
                            'model_type': model_type,
                            'model_name': model_name,
                            'score': score
                        })
            
            # Convert to DataFrame
            df = pd.DataFrame(all_models_data)
            
            # Skip if no data
            if df.empty:
                print(f"  Skipping metric {metric_name}: No data")
                continue
            
            # Get unique model types and ensure specific order with Sentence Transformer first
            model_types_list = list(df['model_type'].unique())
            
            # Custom sort to put 'Sentence Transformer/TF-IDF' first
            if 'Sentence Transformer/TF-IDF' in model_types_list:
                model_types_list.remove('Sentence Transformer/TF-IDF')
                # Sort other model types alphanumerically
                model_types_list.sort()
                # Put Sentence Transformer first
                model_types_list = ['Sentence Transformer/TF-IDF'] + model_types_list
            
            # Calculate statistics for each model type in the desired order
            stats_data = []
            for model_type in model_types_list:
                model_scores = df[df['model_type'] == model_type]['score']
                stats_data.append({
                    'model_type': model_type,
                    'min': model_scores.min(),
                    'q1': model_scores.quantile(0.25),
                    'median': model_scores.median(),
                    'mean': model_scores.mean(),
                    'q3': model_scores.quantile(0.75),
                    'max': model_scores.max(),
                    'std': model_scores.std()
                })
            
            # Convert to DataFrame
            stats_df = pd.DataFrame(stats_data)
            
            # Create figure with subplots explicitly
            fig = plt.figure(figsize=(12, 10))
            
            # Create subplots with specific heights and spacing
            # Use larger gridspec to have more control over spacing
            gs = plt.GridSpec(20, 1, figure=fig)
            
            # Boxplot occupies the top 14 rows
            ax_box = fig.add_subplot(gs[0:14, 0])
            
            # Stats table occupies the bottom 6 rows
            ax_heatmap = fig.add_subplot(gs[14:20, 0])
            
            # Create boxplot with custom colors for each model type
            for i, model_type in enumerate(model_types_list):
                model_data = df[df['model_type'] == model_type]
                
                # Calculate position
                positions = [i]
                
                # Create boxplot for this model type
                boxplot = ax_box.boxplot(
                    model_data['score'],
                    positions=positions,
                    patch_artist=True,
                    widths=0.6,
                    showcaps=True,
                    showfliers=False,
                    # Make box outlines thinner
                    boxprops={'linewidth': 0.8},
                    # Make whiskers thinner
                    whiskerprops={'linewidth': 0.8},
                    # Make caps thinner
                    capprops={'linewidth': 0.8},
                    # Make median line thinner
                    medianprops={'linewidth': 0.8}
                )
                
                # Set boxplot colors
                for box in boxplot['boxes']:
                    box.set(facecolor=color_map.get(model_type, 'gray'))
                
                # Number of points for this model type
                n_points = len(model_data)
                
                # Create jittered x positions - similar to how it's done in the reference notebook
                # Set jitter width relative to the boxplot width
                jitter_width = 0.3
                
                # Generate random offsets for each point
                jittered_x = np.full(n_points, i) + np.random.uniform(-jitter_width, jitter_width, size=n_points)
                
                # Add individual data points with jitter
                ax_box.scatter(
                    jittered_x,
                    model_data['score'],
                    color='black',
                    alpha=0.5,
                    s=20,
                    zorder=3
                )
            
            # Set x-ticks to model types but don't show labels (they'll be in the heatmap)
            ax_box.set_xticks(range(len(model_types_list)))
            ax_box.set_xticklabels([''] * len(model_types_list))
            
            # Make tick marks thinner
            ax_box.tick_params(axis='both', width=0.8, length=4, pad=4)
            
            # Remove bottom spacing by hiding x-axis
            ax_box.spines['bottom'].set_visible(False)
            ax_box.tick_params(axis='x', which='both', bottom=False, labelbottom=False)
            
            # Make all spines thinner
            for spine in ax_box.spines.values():
                spine.set_linewidth(0.8)
            
            # Customize the boxplot
            ax_box.set_title(f'Project: {project} - {metric_name}', fontsize=14)
            ax_box.set_xlabel('')  # Remove x-axis label
            ax_box.set_ylabel('Score', fontsize=12)
            ax_box.grid(axis='y', linestyle='--', alpha=0.7, linewidth=0.6)
            ax_box.set_ylim(0, 1.0)  # Metrics are typically in range [0, 1]
            
            # Add a horizontal line at y=0.5 as a reference
            ax_box.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, linewidth=0.8)
            
            # Prepare data for heatmap
            heatmap_data = pd.DataFrame(index=['Min', 'Q1', 'Median', 'Mean', 'Q3', 'Max', 'Std Dev'])
            
            # Add model data in the custom order
            for model_type in model_types_list:
                model_row = stats_df[stats_df['model_type'] == model_type].iloc[0]
                heatmap_data[model_type] = [
                    model_row['min'], model_row['q1'], model_row['median'], 
                    model_row['mean'], model_row['q3'], model_row['max'], model_row['std']
                ]
            
            # Create formatted version for display
            formatted_data = {}
            for col in heatmap_data.columns:
                formatted_data[col] = []
                for i, val in enumerate(heatmap_data[col]):
                    if i == 6:  # Std Dev row
                        formatted_data[col].append(f"{val:.3f}")
                    else:
                        formatted_data[col].append(f"{val:.3f}")
            
            formatted_df = pd.DataFrame(formatted_data, index=heatmap_data.index)
            
            # Create the heatmap with custom colormap
            sns.heatmap(
                heatmap_data,
                annot=formatted_df,
                fmt="",
                cmap="YlGnBu",
                linewidths=0.5,
                linecolor='lightgray',
                cbar=False,
                ax=ax_heatmap,
                vmin=0,
                vmax=1.0  # Set maximum value for color scaling (most metrics are 0-1)
            )
            
            # Make heatmap tick marks thinner
            ax_heatmap.tick_params(axis='both', width=0.8, length=4)
            
            # Customize heatmap appearance
            ax_heatmap.set_title('')  # Remove redundant title
            ax_heatmap.set_xticklabels(ax_heatmap.get_xticklabels(), rotation=45, ha='right')
            
            # Custom color for std dev row
            cells = ax_heatmap.get_children()
            std_dev_row_idx = 6  # Std Dev is the last row
            
            # Filter for rectangle patches (cells) that are in the std dev row
            for i, cell in enumerate(cells):
                if hasattr(cell, 'get_xy'):  # Check if it's a patch with coordinates
                    y = cell.get_xy()[1]
                    # If this cell is in the std dev row
                    if abs(y - std_dev_row_idx) < 0.1:
                        cell.set_facecolor('#f5f5f5')  # Light gray for std dev row
            
            # Adjust layout to reduce space between plots
            plt.subplots_adjust(hspace=1.0)  # Minimal space between subplots
            
            # Save figure with bbox_inches to ensure all elements are included
            plt.savefig(f"{project}_{metric_name.replace(' ', '_')}_with_points.png", dpi=300, bbox_inches='tight')
            plt.show()
            
            print(f"  Generated visualization for metric: {metric_name}")
        
        print(f"Completed visualizations for project: {project}")

# Import required for patches
from matplotlib.patches import Patch

# Generate visualizations with individual data points
create_metric_visualizations_with_points(raw_model_data)

In [ ]:
# Cell [5] - Combined Multi-Project Performance Comparison
# Purpose: Create comprehensive cross-project analysis comparing model types and performance
# Dependencies: pandas, matplotlib, seaborn, numpy
# Breadcrumbs: Visualization -> Cross-Project Analysis

def create_multi_project_comparison(raw_model_data):
    """
    Create a comprehensive comparison of model types across all projects with statistical analysis
    
    Parameters:
        raw_model_data (dict): Nested dictionary containing organized model data from extract_raw_model_data()
                              Structure: {project: {metric: {model_type: {model_name: [scores]}}}}
                              
    Returns:
        None: Displays multiple visualizations and saves PNG files
        
    Notes:
        - Focuses on key metrics: F1 Score, F2 Score, Matthews Correlation
        - Creates grouped boxplots comparing model types across projects
        - Generates detailed statistical tables for each metric
        - Uses consistent color mapping with Sentence Transformer/TF-IDF prioritized
        - Includes jittered individual data points for transparency
        - Saves separate high-resolution images for each metric comparison
    """
    # Import numpy for jitter function
    import numpy as np
    
    if not raw_model_data:
        print("No data available for comparison.")
        return
    
    # Select key metrics for comparison
    key_metrics = ['F1 Score', 'F2 Score', 'Matthews Correlation']
    
    # Create dynamic color map based on actual model types in the data
    all_model_types = set()
    for project, metrics in raw_model_data.items():
        for metric, model_types in metrics.items():
            all_model_types.update(model_types.keys())
    
    # Define color palette
    color_palette = ['gold', 'skyblue', 'lightgreen', 'lightpink', 'mediumpurple', 'lightcoral', 'lightblue', 'lightsalmon']
    
    # Create color map dynamically
    color_map = {}
    for i, model_type in enumerate(all_model_types):
        color_map[model_type] = color_palette[i % len(color_palette)]
    
    # Count projects with data
    projects_with_data = []
    for project, metrics in raw_model_data.items():
        has_key_metric = any(metric in metrics for metric in key_metrics)
        if has_key_metric:
            projects_with_data.append(project)
    
    if not projects_with_data:
        print("No projects have data for key metrics.")
        return
        
    print(f"\nCreating multi-project comparison for {len(projects_with_data)} projects")
    print("-" * 80)
    
    # Prepare data for each key metric
    for metric_name in key_metrics:
        print(f"Processing metric: {metric_name}")
        
        # Collect data from all projects
        comparison_data = []
        
        for project in projects_with_data:
            if metric_name not in raw_model_data[project]:
                continue
                
            model_types = raw_model_data[project][metric_name]
            
            for model_type, models in model_types.items():
                for model_name, scores in models.items():
                    # Add each score as a separate row
                    for score in scores:
                        comparison_data.append({
                            'project': project,
                            'model_type': model_type,
                            'model_name': model_name,
                            'score': score
                        })
        
        # Skip if not enough data
        if len(comparison_data) < 5:
            print(f"  Skipping {metric_name}: Not enough data")
            continue
            
        # Convert to DataFrame
        df = pd.DataFrame(comparison_data)
        
        # Create figure for this metric
        plt.figure(figsize=(15, 8))
        
        # Order projects alphabetically
        projects_ordered = sorted(df['project'].unique())
        
        # Get unique model types and ensure specific order with Sentence Transformer first
        model_types_list = list(df['model_type'].unique())
        if 'Sentence Transformer/TF-IDF' in model_types_list:
            model_types_list.remove('Sentence Transformer/TF-IDF')
            model_types_list.sort()
            model_types_list = ['Sentence Transformer/TF-IDF'] + model_types_list
        
        # Create custom grouped boxplot
        for p_idx, project in enumerate(projects_ordered):
            project_data = df[df['project'] == project]
            
            # Base position for this project
            base_pos = p_idx
            
            # Number of model types for spacing
            n_models = len(model_types_list)
            
            # Width of a model group within a project
            model_width = 0.8 / n_models
            
            # Plot each model type as a separate boxplot
            for m_idx, model_type in enumerate(model_types_list):
                model_project_data = project_data[project_data['model_type'] == model_type]
                
                if not model_project_data.empty:
                    # Calculate position for this model within the project
                    pos = base_pos - 0.4 + model_width * (m_idx + 0.5)
                    
                    # Create boxplot
                    boxplot = plt.boxplot(
                        model_project_data['score'],
                        positions=[pos],
                        patch_artist=True,
                        widths=model_width * 0.8,
                        showcaps=True,
                        showfliers=False,
                        boxprops={'linewidth': 0.8},
                        whiskerprops={'linewidth': 0.8},
                        capprops={'linewidth': 0.8},
                        medianprops={'linewidth': 0.8, 'color': 'black'}
                    )
                    
                    # Set color
                    for box in boxplot['boxes']:
                        box.set(facecolor=color_map.get(model_type, 'gray'))
                    
                    # Add jittered points
                    n_points = len(model_project_data)
                    jitter_width = model_width * 0.3
                    jittered_x = np.full(n_points, pos) + np.random.uniform(-jitter_width, jitter_width, size=n_points)
                    
                    plt.scatter(
                        jittered_x, 
                        model_project_data['score'], 
                        color='black',
                        alpha=0.5,
                        s=20,
                        zorder=3
                    )
        
        # Create custom legend
        legend_elements = [
            Patch(facecolor=color_map.get(model_type, 'gray'), label=model_type)
            for model_type in model_types_list
        ]
        plt.legend(handles=legend_elements, title='Model Type')
        
        # Set x-ticks at the middle of each project group
        plt.xticks(range(len(projects_ordered)), projects_ordered)
        
        # Customize the plot
        plt.title(f'Cross-Project Comparison: {metric_name}', fontsize=16)
        plt.xlabel('Project', fontsize=14)
        plt.ylabel('Score', fontsize=14)
        plt.grid(axis='y', linestyle='--', alpha=0.7, linewidth=0.6)
        plt.ylim(0, 1.05)
        
        # Add a reference line at y=0.5
        plt.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, linewidth=0.8)
        
        # Make tick marks and spines thinner
        ax = plt.gca()
        ax.tick_params(axis='both', width=0.8, length=4)
        for spine in ax.spines.values():
            spine.set_linewidth(0.8)
        
        # Calculate statistics table
        stats_data = []
        for project in projects_ordered:
            for model_type in model_types_list:
                subset = df[(df['project'] == project) & (df['model_type'] == model_type)]
                if not subset.empty:
                    stats_data.append({
                        'project': project,
                        'model_type': model_type,
                        'mean': subset['score'].mean(),
                        'median': subset['score'].median(),
                        'min': subset['score'].min(),
                        'max': subset['score'].max(),
                        'count': len(subset)
                    })
        
        # Create statistics table
        stats_df = pd.DataFrame(stats_data)
        
        # Sort stats by project and model_type
        stats_df = stats_df.sort_values(['project', 'model_type'])
        
        # Create stats table as a separate figure
        fig_stats, ax_stats = plt.subplots(figsize=(12, len(stats_df) * 0.4 + 1))
        ax_stats.axis('tight')
        ax_stats.axis('off')
        
        table = ax_stats.table(
            cellText=stats_df[['project', 'model_type', 'mean', 'median', 'min', 'max', 'count']].round(3).values,
            colLabels=['Project', 'Model Type', 'Mean', 'Median', 'Min', 'Max', 'Count'],
            loc='center',
            cellLoc='center'
        )
        
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 1.2)
        
        # Set title for stats table
        plt.suptitle(f'Statistics for {metric_name}', fontsize=14)
        plt.tight_layout()
        
        # Save figures
        plt.savefig(f"multi_project_{metric_name.replace(' ', '_')}.png", dpi=300, bbox_inches='tight')
        fig_stats.savefig(f"stats_{metric_name.replace(' ', '_')}.png", dpi=300, bbox_inches='tight')
        
        plt.show()
        plt.close(fig_stats)
        
        print(f"  Generated comparison for {metric_name}")

# Import required for patches
from matplotlib.patches import Patch

# Create multi-project comparison
create_multi_project_comparison(raw_model_data)

In [ ]:
# Cell [6] - LLM Improvement Analysis
# Purpose: Comprehensive analysis of LLM performance improvements over baseline techniques
# Dependencies: pandas, matplotlib, numpy
# Breadcrumbs: Analysis -> LLM Performance Evaluation

def analyze_llm_improvements(raw_model_data):
    """
    Analyze and quantify how LLMs improve over Sentence Transformer/TF-IDF baseline approaches
    
    Parameters:
        raw_model_data (dict): Nested dictionary containing organized model data from extract_raw_model_data()
                              Structure: {project: {metric: {model_type: {model_name: [scores]}}}}
                              
    Returns:
        pd.DataFrame: DataFrame containing improvement analysis results with columns for:
                     - project, metric, llm_type, st_score, llm_score
                     - abs_improvement, rel_improvement, sample_sizes
                     
    Notes:
        - Compares LLM performance against Sentence Transformer/TF-IDF baseline
        - Calculates both absolute and relative improvements
        - Focuses on key metrics: Accuracy, Balanced Accuracy, Precision, Recall, F1, F2, Matthews Correlation
        - Groups analysis by metric, LLM type, and project
        - Generates comprehensive visualizations showing improvement patterns
        - Creates heatmaps for cross-project comparison
        - Maintains metric order consistency with other analysis notebooks
    """
    if not raw_model_data:
        print("No data available for analysis.")
        return
    
    # Set of key metrics in the same order as in Meta Judge Analysis Notebook
    key_metrics = ['Accuracy', 'Balanced Accuracy', 'Precision', 'Recall', 
                  'F1 Score', 'F2 Score', 'Matthews Correlation']
    
    # Collect all data for comparison
    improvement_data = []
    
    # Get all model types from data
    all_model_types = set()
    for project, metrics in raw_model_data.items():
        for metric_name, model_types in metrics.items():
            all_model_types.update(model_types.keys())
    
    # Separate LLM model types from Sentence Transformer/TF-IDF
    llm_model_types = [model for model in all_model_types if model != 'Sentence Transformer/TF-IDF']
    
    # Check if we have both model types for comparison
    if 'Sentence Transformer/TF-IDF' not in all_model_types or not llm_model_types:
        print("Both Sentence Transformer/TF-IDF and LLM models are required for comparison.")
        return
    
    print("\nANALYZING LLM IMPROVEMENTS OVER SENTENCE TRANSFORMER/TF-IDF")
    print("=" * 80)
    
    # Process data for each project and metric
    for project, metrics in raw_model_data.items():
        for metric_name, model_types in metrics.items():
            if metric_name not in key_metrics:
                continue
                
            # Check if we have both model types for this metric
            if 'Sentence Transformer/TF-IDF' not in model_types:
                continue
                
            # Get Sentence Transformer/TF-IDF scores
            st_scores = []
            for model_name, scores in model_types['Sentence Transformer/TF-IDF'].items():
                st_scores.extend(scores)
            
            if not st_scores:
                continue
                
            # Calculate average Sentence Transformer score
            avg_st_score = sum(st_scores) / len(st_scores)
            
            # Compare with each LLM model type
            for model_type in llm_model_types:
                if model_type not in model_types:
                    continue
                    
                llm_scores = []
                for model_name, scores in model_types[model_type].items():
                    llm_scores.extend(scores)
                
                if not llm_scores:
                    continue
                    
                # Calculate average LLM score
                avg_llm_score = sum(llm_scores) / len(llm_scores)
                
                # Calculate improvement
                abs_improvement = avg_llm_score - avg_st_score
                rel_improvement = (abs_improvement / avg_st_score) * 100 if avg_st_score > 0 else float('inf')
                
                # Store the comparison
                improvement_data.append({
                    'project': project,
                    'metric': metric_name,
                    'llm_type': model_type,
                    'st_score': avg_st_score,
                    'llm_score': avg_llm_score,
                    'abs_improvement': abs_improvement,
                    'rel_improvement': rel_improvement,
                    'st_sample_size': len(st_scores),
                    'llm_sample_size': len(llm_scores)
                })
    
    if not improvement_data:
        print("No comparable data between Sentence Transformer/TF-IDF and LLM models.")
        return
        
    # Convert to DataFrame
    improvement_df = pd.DataFrame(improvement_data)
    
    # Sort by absolute improvement (descending)
    improvement_df = improvement_df.sort_values('abs_improvement', ascending=False)
    
    # Display overall summary
    print("\n1. OVERALL IMPROVEMENT SUMMARY")
    print("-" * 70)
    
    avg_overall_improvement = improvement_df['abs_improvement'].mean()
    avg_rel_improvement = improvement_df['rel_improvement'].mean()
    
    print(f"Average absolute improvement across all metrics: {avg_overall_improvement:.3f}")
    print(f"Average relative improvement: {avg_rel_improvement:.1f}%")
    
    # Count cases where LLMs outperform ST/TF-IDF
    better_count = (improvement_df['abs_improvement'] > 0).sum()
    worse_count = (improvement_df['abs_improvement'] <= 0).sum()
    
    print(f"LLMs outperform ST/TF-IDF in {better_count} of {len(improvement_df)} cases ({better_count/len(improvement_df)*100:.1f}%)")
    print(f"LLMs underperform ST/TF-IDF in {worse_count} of {len(improvement_df)} cases ({worse_count/len(improvement_df)*100:.1f}%)")
    
    # Analyze by metric, maintaining the same order as Meta Judge notebook
    print("\n2. IMPROVEMENT BY METRIC")
    print("-" * 70)
    
    # Create a categorical type with our preferred order
    metric_cat_type = pd.CategoricalDtype(categories=key_metrics, ordered=True)
    improvement_df['metric_ordered'] = improvement_df['metric'].astype(metric_cat_type)
    
    # Group by the ordered metric and calculate means
    metric_improvement = improvement_df.groupby('metric_ordered')['abs_improvement'].agg(['mean', 'count'])
    
    for metric, row in metric_improvement.iterrows():
        metric_data = improvement_df[improvement_df['metric'] == metric]
        better_rate = (metric_data['abs_improvement'] > 0).mean() * 100
        print(f"  {metric}: {row['mean']:.3f} avg improvement ({better_rate:.1f}% of cases show improvement)")
    
    # Analyze by LLM type
    print("\n3. IMPROVEMENT BY LARGE LANGUAGE MODEL NAME")
    print("-" * 70)
    
    llm_improvement = improvement_df.groupby('llm_type')['abs_improvement'].agg(['mean', 'count'])
    llm_improvement = llm_improvement.sort_values('mean', ascending=False)
    
    for llm_type, row in llm_improvement.iterrows():
        llm_data = improvement_df[improvement_df['llm_type'] == llm_type]
        better_rate = (llm_data['abs_improvement'] > 0).mean() * 100
        print(f"  {llm_type}: {row['mean']:.3f} avg improvement ({better_rate:.1f}% of cases show improvement)")
    
    # Analyze by project
    print("\n4. IMPROVEMENT BY PROJECT")
    print("-" * 70)
    
    project_improvement = improvement_df.groupby('project')['abs_improvement'].agg(['mean', 'count'])
    project_improvement = project_improvement.sort_values('mean', ascending=False)
    
    for project, row in project_improvement.iterrows():
        project_data = improvement_df[improvement_df['project'] == project]
        better_rate = (project_data['abs_improvement'] > 0).mean() * 100
        print(f"  {project}: {row['mean']:.3f} avg improvement ({better_rate:.1f}% of cases show improvement)")
    
    # Create visualization of improvements for all projects combined
    if len(improvement_df['project'].unique()) > 1:
        # Create combined visualization
        fig, ax = plt.subplots(figsize=(14, 7))
        
        # Create a pivot table for ordered metrics
        pivot_data = improvement_df.pivot_table(
            index='metric_ordered', 
            columns='llm_type', 
            values='abs_improvement', 
            aggfunc='mean'
        )
        
        # Plot the data
        pivot_data.plot(
            kind='bar',
            ax=ax,
            width=0.8,
            alpha=0.7,
            edgecolor='black',
            linewidth=0.5
        )
        
        # Add grid lines
        ax.grid(axis='y', linestyle='--', alpha=0.7, linewidth=0.6)
        
        # Set y-axis limit to 0.50
        ax.set_ylim(top=0.50)
        
        # Add title with extra padding to make room for legend
        ax.set_title('Combined Projects: LLM Improvement over Sentence Transformer/TF-IDF by Metric', 
                     fontsize=16, pad=40)  # Add extra padding below title
        
        # Split y-axis label into two lines
        ax.set_ylabel('Absolute Improvement\n(LLM - ST/TF-IDF)', fontsize=14)
        ax.set_xlabel('Metric', fontsize=14)
        
        # Add a zero line for reference
        ax.axhline(y=0, color='red', linestyle='-', alpha=0.3, linewidth=1)
        
        # Rotate x-axis labels for better readability
        plt.xticks(rotation=45, ha='right')
        
        # Make tick marks thinner
        ax.tick_params(axis='both', width=0.8, length=4)
        
        # Make all spines thinner
        for spine in ax.spines.values():
            spine.set_linewidth(0.8)
        
        # Position the legend below the title but above the chart
        legend = ax.legend(
            title='Large Language Model Name',
            loc='upper center',
            bbox_to_anchor=(0.5, 1.01),  # Position just below the title
            ncol=3,
            fontsize=12,
            frameon=True  # Add a frame around the legend
        )
        
        # Make sure legend title is visible
        legend.get_title().set_fontsize(12)
        
        # Increase top margin to ensure nothing gets cut off
        plt.subplots_adjust(top=0.85)
        
        # Save the figure
        plt.savefig("combined_llm_improvement_by_metric.png", dpi=300, bbox_inches='tight')
        plt.show()
    
    # Create project-specific visualizations with ordered metrics
    for project_name in improvement_df['project'].unique():
        project_df = improvement_df[improvement_df['project'] == project_name]
        
        fig, ax = plt.subplots(figsize=(14, 7))
        
        # Create a grouped bar chart of improvements by metric and LLM type
        pivot_data = project_df.pivot_table(
            index='metric_ordered', 
            columns='llm_type', 
            values='abs_improvement', 
            aggfunc='mean'
        )
        
        # Plot the data
        pivot_data.plot(
            kind='bar',
            ax=ax,
            width=0.8,
            alpha=0.7,
            edgecolor='black',
            linewidth=0.5
        )
        
        # Add grid lines
        ax.grid(axis='y', linestyle='--', alpha=0.7, linewidth=0.6)
        
        # Set y-axis limit to 0.50
        ax.set_ylim(top=0.50)
        
        # Add title with extra padding to make room for legend
        ax.set_title(f'Project: {project_name} - LLM Improvement over Sentence Transformer/TF-IDF by Metric', 
                     fontsize=16, pad=40)  # Add extra padding below title
        
        # Split y-axis label into two lines
        ax.set_ylabel('Absolute Improvement\n(LLM - ST/TF-IDF)', fontsize=14)
        ax.set_xlabel('Metric', fontsize=14)
        
        # Add a zero line for reference
        ax.axhline(y=0, color='red', linestyle='-', alpha=0.3, linewidth=1)
        
        # Rotate x-axis labels for better readability
        plt.xticks(rotation=45, ha='right')
        
        # Make tick marks thinner
        ax.tick_params(axis='both', width=0.8, length=4)
        
        # Make all spines thinner
        for spine in ax.spines.values():
            spine.set_linewidth(0.8)
        
        # Position the legend below the title but above the chart
        legend = ax.legend(
            title='Large Language Model Name',
            loc='upper center',
            bbox_to_anchor=(0.5, 1.01),  # Position just below the title
            ncol=3,
            fontsize=12,
            frameon=True  # Add a frame around the legend
        )
        
        # Make sure legend title is visible
        legend.get_title().set_fontsize(12)
        
        # Increase top margin to ensure nothing gets cut off
        plt.subplots_adjust(top=0.85)
        
        # Save the figure with project name in filename
        plt.savefig(f"{project_name}_llm_improvement_by_metric.png", dpi=300, bbox_inches='tight')
        plt.show()
    
    # Create a heatmap of improvements by project and metric with ordered metrics
    plt.figure(figsize=(14, 5))
    
    # Pivot the data to create a project × metric table of improvements
    # Using the ordered metrics
    heatmap_data = improvement_df.pivot_table(
        index='project', 
        columns='metric_ordered', 
        values='abs_improvement', 
        aggfunc='mean'
    )
    
    # Create the heatmap
    sns.heatmap(
        heatmap_data,
        cmap='RdYlGn',  # Red for negative, green for positive
        center=0,       # Center the colormap at 0
        annot=True,     # Show the values
        fmt='.3f',      # Format as 3 decimal places
        linewidths=0.5,
        cbar_kws={'label': 'Improvement (LLM - ST/TF-IDF)'}
    )
    
    # Add title
    plt.title('LLM Improvement over Sentence Transformer/TF-IDF by Project and Metric', fontsize=16)
    
    # Rotate x-axis labels
    plt.xticks(rotation=45, ha='right')
    
    # Save the figure
    plt.savefig("improvement_heatmap.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    return improvement_df

# Analyze LLM improvements over Sentence Transformer/TF-IDF
improvement_results = analyze_llm_improvements(raw_model_data)

In [ ]:
# Cell [7] - Retrieve Confusion Matrix Data for Model Comparison
# Purpose: Query Neo4j database for comprehensive confusion matrix data across model types
# Dependencies: neo4j, pandas, json, logging
# Breadcrumbs: Data Acquisition -> Confusion Matrix Retrieval

def retrieve_confusion_matrix_data(driver):
    """
    Retrieve comprehensive confusion matrix data (TP, FP, FN, TN) for sentence transformer and LLM models
    
    Parameters:
        driver: Neo4j driver connection for database access
        
    Returns:
        pd.DataFrame: Combined DataFrame with confusion matrix metrics across all models containing:
                     - project_name, model_type, model_name, metric (TP/FP/FN/TN)
                     - value, threshold, created_at
                     Returns empty DataFrame if no data found or on error.
                     
    Notes:
        - Retrieves data from MetricsAnalysis nodes with results field
        - Parses JSON results to extract confusion matrix components
        - Creates long-format DataFrame for easy analysis and visualization
        - Handles multiple model types and projects in single query
        - Includes threshold information for reproducibility
    """
    try:
        # Query to retrieve metrics analysis data for both model types
        query = """
        MATCH (p:Project)-[r:HAS_METRICS_ANALYSIS]->(m:MetricsAnalysis)
        WHERE m.results IS NOT NULL
        RETURN 
            p.name as project_name,
            m.model_type as model_type,
            m.results as results_json,
            m.created_at as created_at
        ORDER BY p.name, m.model_type
        """
        
        with driver.session() as session:
            results = session.run(query).data()
            
            if not results:
                logger.warning("No confusion matrix data found in Neo4j")
                return pd.DataFrame()
                
            print(f"Retrieved metrics data for {len(results)} analysis records")
            
            # Process results to extract confusion matrix data
            combined_data = []
            
            for record in results:
                project_name = record['project_name']
                model_type = record['model_type']
                created_at = record['created_at']
                
                # Parse the results JSON
                if record['results_json'] and isinstance(record['results_json'], str):
                    try:
                        results_data = json.loads(record['results_json'])
                        
                        # Extract confusion matrix data for each model
                        for model_name, metrics in results_data.items():
                            # Create a row for each confusion matrix component
                            for metric in ['true_positives', 'false_positives', 'false_negatives', 'true_negatives']:
                                if metric in metrics:
                                    combined_data.append({
                                        'project_name': project_name,
                                        'model_type': model_type,
                                        'model_name': model_name,
                                        'metric': metric,
                                        'value': metrics[metric],
                                        'threshold': metrics.get('threshold', None),
                                        'created_at': created_at
                                    })
                    except json.JSONDecodeError as e:
                        print(f"Error parsing JSON for {project_name}, {model_type}: {e}")
                        continue
            
            # Convert to DataFrame
            confusion_df = pd.DataFrame(combined_data)
            
            if confusion_df.empty:
                print("No confusion matrix data could be extracted")
                return pd.DataFrame()
            
            # Display summary of retrieved data
            project_count = confusion_df['project_name'].nunique()
            model_type_count = confusion_df['model_type'].nunique()
            model_count = confusion_df['model_name'].nunique()
            
            print(f"Extracted confusion matrix data for {project_count} projects, {model_type_count} model types, and {model_count} models")
            
            # Count metrics by type
            metric_counts = confusion_df['metric'].value_counts()
            print("\nMetric counts:")
            for metric, count in metric_counts.items():
                print(f"  {metric}: {count}")
            
            # Count by model type
            model_type_counts = confusion_df['model_type'].value_counts()
            print("\nModel type counts:")
            for model_type, count in model_type_counts.items():
                print(f"  {model_type}: {count}")
            
            return confusion_df
    
    except Exception as e:
        logger.error(f"Error retrieving confusion matrix data: {str(e)}")
        logger.error("Exception details:", exc_info=True)
        print(f"Error retrieving confusion matrix data: {str(e)}")
        return pd.DataFrame()

# Retrieve confusion matrix data from Neo4j
confusion_matrix_df = retrieve_confusion_matrix_data(driver)

# Display sample of the data
if not confusion_matrix_df.empty:
    display(confusion_matrix_df.head(10))
    
    # Show distribution of metrics by project and model type
    print("\nProjects and model types in the dataset:")
    for project in confusion_matrix_df['project_name'].unique():
        project_data = confusion_matrix_df[confusion_matrix_df['project_name'] == project]
        print(f"\n  Project: {project}")
        for model_type in project_data['model_type'].unique():
            model_type_data = project_data[project_data['model_type'] == model_type]
            models_count = model_type_data['model_name'].nunique()
            print(f"    {model_type}: {models_count} models")
else:
    print("No confusion matrix data available for visualization")

In [ ]:
# Cell [8] - Create Combined Whisker Charts for Confusion Matrix Metrics
# Purpose: Generate comprehensive whisker chart visualizations for confusion matrix analysis
# Dependencies: pandas, matplotlib, seaborn, numpy
# Breadcrumbs: Visualization -> Confusion Matrix Analysis

def create_confusion_matrix_whisker_charts(confusion_df):
    """
    Create comprehensive whisker charts comparing confusion matrix metrics across model types
    
    Parameters:
        confusion_df (pd.DataFrame): DataFrame containing confusion matrix metrics with columns:
                                    - project_name, model_type, model_name, metric, value
                                    
    Returns:
        None: Displays visualizations directly using matplotlib/seaborn
        
    Notes:
        - Creates separate subplot grids for each project (2x2 layout for TP, FP, FN, TN)
        - Uses human-readable labels for confusion matrix components
        - Includes individual data points with jitter for detailed analysis
        - Generates combined view with all metrics side-by-side for comparison
        - Handles large datasets with appropriate scaling and positioning
        - Prints statistical summaries for each model type and metric
    """
    if confusion_df.empty:
        print("No data available for visualization")
        return
    
    # Define human-readable names for metrics
    metric_labels = {
        'true_positives': 'True Positives (TP)',
        'false_positives': 'False Positives (FP)', 
        'false_negatives': 'False Negatives (FN)',
        'true_negatives': 'True Negatives (TN)'
    }
    
    # Get list of all projects
    projects = confusion_df['project_name'].unique()
    
    # Process each project separately
    for project in projects:
        project_data = confusion_df[confusion_df['project_name'] == project]
        
        # Skip if insufficient data
        if project_data.empty:
            continue
            
        print(f"\nProcessing visualization for project: {project}")
        
        # Get model types for this project
        model_types = project_data['model_type'].unique()
        print(f"  Model types: {', '.join(model_types)}")
        
        # Create figure for this project - one subplot for each metric
        fig, axes = plt.subplots(2, 2, figsize=(20, 16))
        axes = axes.flatten()
        
        # Process each confusion matrix metric
        for i, metric in enumerate(['true_positives', 'false_positives', 'false_negatives', 'true_negatives']):
            ax = axes[i]
            
            # Filter data for this metric
            metric_data = project_data[project_data['metric'] == metric]
            
            if metric_data.empty:
                ax.text(0.5, 0.5, f"No data for {metric_labels[metric]}", 
                       horizontalalignment='center', fontsize=14)
                print(f"  No data available for {metric_labels[metric]}")
                continue
            
            # Calculate positions for boxplots
            model_count = len(model_types)
            positions = np.arange(model_count)
            
            # Create a boxplot for each model type
            boxplot_data = []
            boxplot_positions = []
            model_type_names = []
            
            # Print statistics for this metric
            print(f"\n  Statistics for {metric_labels[metric]}:")
            
            for pos, model_type in enumerate(model_types):
                # Filter data for this model type
                model_type_data = metric_data[metric_data['model_type'] == model_type]
                
                if not model_type_data.empty:
                    boxplot_data.append(model_type_data['value'])
                    boxplot_positions.append(pos)
                    
                    # Format model type name for display
                    if model_type == 'sentence_transformer_tf_idf':
                        display_name = 'Sentence Transformer/TF-IDF'
                    else:
                        display_name = model_type.upper() if model_type == 'llm' else model_type
                    
                    model_type_names.append(display_name)
                    
                    # Calculate and print statistics
                    mean_val = model_type_data['value'].mean()
                    median_val = model_type_data['value'].median()
                    min_val = model_type_data['value'].min()
                    max_val = model_type_data['value'].max()
                    count = len(model_type_data)
                    print(f"    {display_name}: Mean={mean_val:.1f}, Median={median_val:.1f}, Min={min_val}, Max={max_val}, Count={count}")
            
            # Create boxplot with uniform color
            bplot = ax.boxplot(
                boxplot_data, 
                positions=boxplot_positions, 
                patch_artist=True, 
                widths=0.6,
                showfliers=False
            )
            
            # Set uniform color for all boxes
            for patch in bplot['boxes']:
                patch.set_facecolor('lightgray')
            
            # Add individual data points with jitter
            for j, (model_type, pos) in enumerate(zip(model_types, positions)):
                model_type_data = metric_data[metric_data['model_type'] == model_type]
                
                if not model_type_data.empty:
                    # Create jittered x positions
                    x = np.random.normal(pos, 0.1, size=len(model_type_data))
                    
                    # Plot points
                    ax.scatter(
                        x, 
                        model_type_data['value'], 
                        color='black', 
                        alpha=0.5, 
                        s=30,
                        zorder=10
                    )
            
            # Set axis labels and title
            ax.set_title(metric_labels[metric], fontsize=16)
            ax.set_ylabel('Count', fontsize=14)
            ax.set_xticks(positions)
            ax.set_xticklabels(model_type_names, rotation=45, ha='right', fontsize=12)
            
            # Add grid
            ax.grid(axis='y', linestyle='--', alpha=0.7)
            
        # Add overall title
        plt.suptitle(f'Confusion Matrix Metrics Comparison for Project: {project}', fontsize=20)
        
        plt.tight_layout()
        plt.subplots_adjust(top=0.92)  # Make room for title
        plt.show()
        
        # Create a single boxplot with all metrics side by side (alternative view)
        print(f"\nCreating combined view for project: {project}")
        
        # Reshape data for combined visualization
        plot_data = []
        for metric in ['true_positives', 'false_positives', 'false_negatives', 'true_negatives']:
            metric_data = project_data[project_data['metric'] == metric]
            
            for _, row in metric_data.iterrows():
                model_name = 'Sentence Transformer/TF-IDF' if row['model_type'] == 'sentence_transformer_tf_idf' else row['model_type'].upper() if row['model_type'] == 'llm' else row['model_type']
                plot_data.append({
                    'metric': metric_labels[metric],
                    'model_type': model_name,
                    'value': row['value'],
                    'model_name': row['model_name']
                })
        
        # Convert to DataFrame
        plot_df = pd.DataFrame(plot_data)
        
        if not plot_df.empty:
            print("  Model types in combined view:")
            for model_type in sorted(plot_df['model_type'].unique()):
                print(f"    - {model_type}")
                
            # Create a figure for manual plotting (avoiding seaborn's hue parameter)
            fig, ax = plt.subplots(figsize=(16, 10))
            
            # Get unique metrics and model types
            unique_metrics = [metric_labels[m] for m in ['true_positives', 'false_positives', 'false_negatives', 'true_negatives']]
            unique_model_types = sorted(plot_df['model_type'].unique())
            
            # Set up positions
            n_metrics = len(unique_metrics)
            n_models = len(unique_model_types)
            
            # Calculate bar width and positions
            metric_width = 0.8
            group_width = metric_width / n_models
            
            # For each metric, create grouped boxplots
            positions = []
            for i, metric in enumerate(unique_metrics):
                metric_data = plot_df[plot_df['metric'] == metric]
                
                # Base position for this metric
                base_pos = i
                
                for j, model_type in enumerate(unique_model_types):
                    model_data = metric_data[metric_data['model_type'] == model_type]
                    
                    if not model_data.empty:
                        # Calculate position for this model within the metric group
                        pos = base_pos - metric_width/2 + (j + 0.5) * group_width
                        positions.append(pos)
                        
                        # Create boxplot
                        bp = ax.boxplot(
                            model_data['value'], 
                            positions=[pos],
                            widths=group_width * 0.8,
                            patch_artist=True,
                            showfliers=False
                        )
                        
                        # Set color to gray
                        for element in ['boxes', 'whiskers', 'caps', 'medians']:
                            plt.setp(bp[element], color='black')
                        for box in bp['boxes']:
                            box.set(facecolor='lightgray')
                            
                        # Add individual points with jitter
                        x = np.random.normal(pos, group_width * 0.1, size=len(model_data))
                        ax.scatter(x, model_data['value'], color='black', alpha=0.5, s=20)
            
            # Set x-ticks at the middle of each metric group
            ax.set_xticks(range(len(unique_metrics)))
            ax.set_xticklabels(unique_metrics, fontsize=12)
            
            plt.title(f'Combined Confusion Matrix Metrics for Project: {project}', fontsize=16)
            plt.ylabel('Count', fontsize=14)
            plt.xlabel('Metric', fontsize=14)
            plt.grid(axis='y', linestyle='--', alpha=0.7)
            plt.tight_layout()
            plt.show()

# Create whisker charts for confusion matrix data
create_confusion_matrix_whisker_charts(confusion_matrix_df)

In [ ]:
# Cell [9] - Analysis of Confusion Matrix Metrics and F2 Performance
# Purpose: Comprehensive analysis of model improvements focusing on F2 optimization and tradeoffs
# Dependencies: pandas, numpy, logging
# Breadcrumbs: Analysis -> Confusion Matrix Performance Evaluation

def analyze_confusion_matrix_improvements(confusion_df):
    """
    Analyze improvements in confusion matrix metrics between model types with F2 score focus
    
    Parameters:
        confusion_df (pd.DataFrame): DataFrame containing confusion matrix metrics with columns:
                                    - project_name, model_type, model_name, metric, value
                                    
    Returns:
        None: Prints comprehensive analysis directly to console
        
    Notes:
        - Focuses on F2 score optimization as primary objective
        - Calculates precision, recall, F1, F2, and accuracy from confusion matrix
        - Analyzes model performance by project with baseline comparisons
        - Identifies best performing models for each metric
        - Evaluates TP/FP tradeoffs critical for F2 optimization
        - Provides actionable recommendations based on performance analysis
        - Suggests areas for further model improvement
    """
    if confusion_df.empty:
        print("No data available for analysis")
        return
    
    print("\n" + "="*80)
    print("ANALYSIS OF MODEL IMPROVEMENTS BASED ON CONFUSION MATRIX METRICS")
    print("="*80)
    print("Focus: F2 score optimization, maximizing TP and TN, minimizing FN, allowing for reasonable FP")
    
    # Get list of all projects
    projects = confusion_df['project_name'].unique()
    
    # Process each project separately
    for project in projects:
        print(f"\n\nPROJECT: {project}")
        print("-"*80)
        
        project_data = confusion_df[confusion_df['project_name'] == project]
        if project_data.empty:
            print("  No data available for this project")
            continue
        
        # Get unique model types for this project
        model_types = project_data['model_type'].unique()
        
        # First, convert the long-format data to wide format for easier analysis
        # We need to pivot the data to have one row per model and columns for TP, FP, FN, TN
        
        # Create a dictionary to store metrics for each model
        model_metrics = {}
        
        # Process each model type
        for model_type in model_types:
            model_type_data = project_data[project_data['model_type'] == model_type]
            
            # Get all models for this type
            models = model_type_data['model_name'].unique()
            
            for model in models:
                model_data = model_type_data[model_type_data['model_name'] == model]
                
                # Initialize metrics dictionary for this model
                if model not in model_metrics:
                    model_metrics[model] = {
                        'model_type': model_type,
                        'display_name': 'Sentence Transformer/TF-IDF' if model_type == 'sentence_transformer_tf_idf' else model_type.upper() if model_type == 'llm' else model_type
                    }
                
                # Extract metrics for this model
                for _, row in model_data.iterrows():
                    metric = row['metric']
                    value = row['value']
                    model_metrics[model][metric] = value
        
        # Convert dictionary to DataFrame
        metrics_df = pd.DataFrame.from_dict(model_metrics, orient='index')
        
        # Fill missing values with 0
        for metric in ['true_positives', 'false_positives', 'false_negatives', 'true_negatives']:
            if metric not in metrics_df.columns:
                metrics_df[metric] = 0
            else:
                metrics_df[metric] = metrics_df[metric].fillna(0)
        
        # Calculate derived metrics
        metrics_df['total_predictions'] = metrics_df['true_positives'] + metrics_df['false_positives'] + \
                                         metrics_df['false_negatives'] + metrics_df['true_negatives']
        
        # Calculate precision, recall, and F2 score
        metrics_df['precision'] = metrics_df['true_positives'] / (metrics_df['true_positives'] + metrics_df['false_positives']).replace(0, np.nan)
        metrics_df['recall'] = metrics_df['true_positives'] / (metrics_df['true_positives'] + metrics_df['false_negatives']).replace(0, np.nan)
        
        # F2 score formula: 5*precision*recall / (4*precision + recall)
        metrics_df['f2_score'] = (5 * metrics_df['precision'] * metrics_df['recall']) / \
                                 (4 * metrics_df['precision'] + metrics_df['recall']).replace(0, np.nan)
        
        # Calculate accuracy
        metrics_df['accuracy'] = (metrics_df['true_positives'] + metrics_df['true_negatives']) / \
                                 metrics_df['total_predictions'].replace(0, np.nan)
        
        # Group by model type and calculate average metrics
        grouped_metrics = metrics_df.groupby('model_type').agg({
            'true_positives': 'mean',
            'false_positives': 'mean',
            'false_negatives': 'mean',
            'true_negatives': 'mean',
            'precision': 'mean',
            'recall': 'mean',
            'f2_score': 'mean',
            'accuracy': 'mean'
        }).reset_index()
        
        # Add display names
        grouped_metrics['display_name'] = grouped_metrics['model_type'].apply(
            lambda x: 'Sentence Transformer/TF-IDF' if x == 'sentence_transformer_tf_idf' 
                     else x.upper() if x == 'llm' else x
        )
        
        # Print summary by model type
        print("\nAVERAGE METRICS BY MODEL TYPE:")
        for _, row in grouped_metrics.iterrows():
            print(f"\n  {row['display_name']}:")
            print(f"    TP: {row['true_positives']:.1f}")
            print(f"    FP: {row['false_positives']:.1f}")
            print(f"    FN: {row['false_negatives']:.1f}")
            print(f"    TN: {row['true_negatives']:.1f}")
            print(f"    Precision: {row['precision']:.3f}")
            print(f"    Recall: {row['recall']:.3f}")
            print(f"    F2 Score: {row['f2_score']:.3f}")
            print(f"    Accuracy: {row['accuracy']:.3f}")
        
        # Find reference model type (sentence transformer)
        st_metrics = grouped_metrics[grouped_metrics['model_type'] == 'sentence_transformer_tf_idf']
        
        if not st_metrics.empty:
            st_row = st_metrics.iloc[0]
            
            # Calculate improvements relative to sentence transformer
            print("\nIMPROVEMENTS RELATIVE TO SENTENCE TRANSFORMER/TF-IDF:")
            
            for _, row in grouped_metrics[grouped_metrics['model_type'] != 'sentence_transformer_tf_idf'].iterrows():
                print(f"\n  {row['display_name']} vs. Sentence Transformer/TF-IDF:")
                
                # Calculate percentage improvements
                tp_change = (row['true_positives'] - st_row['true_positives']) / st_row['true_positives'] * 100 if st_row['true_positives'] > 0 else float('inf')
                fp_change = (row['false_positives'] - st_row['false_positives']) / st_row['false_positives'] * 100 if st_row['false_positives'] > 0 else float('inf')
                fn_change = (row['false_negatives'] - st_row['false_negatives']) / st_row['false_negatives'] * 100 if st_row['false_negatives'] > 0 else float('inf')
                tn_change = (row['true_negatives'] - st_row['true_negatives']) / st_row['true_negatives'] * 100 if st_row['true_negatives'] > 0 else float('inf')
                
                prec_change = (row['precision'] - st_row['precision']) / st_row['precision'] * 100 if st_row['precision'] > 0 else float('inf')
                recall_change = (row['recall'] - st_row['recall']) / st_row['recall'] * 100 if st_row['recall'] > 0 else float('inf')
                f2_change = (row['f2_score'] - st_row['f2_score']) / st_row['f2_score'] * 100 if st_row['f2_score'] > 0 else float('inf')
                acc_change = (row['accuracy'] - st_row['accuracy']) / st_row['accuracy'] * 100 if st_row['accuracy'] > 0 else float('inf')
                
                # Print changes
                print(f"    TP: {row['true_positives']:.1f} ({tp_change:+.1f}%)")
                print(f"    FP: {row['false_positives']:.1f} ({fp_change:+.1f}%)")
                print(f"    FN: {row['false_negatives']:.1f} ({fn_change:+.1f}%)")
                print(f"    TN: {row['true_negatives']:.1f} ({tn_change:+.1f}%)")
                print(f"    Precision: {row['precision']:.3f} ({prec_change:+.1f}%)")
                print(f"    Recall: {row['recall']:.3f} ({recall_change:+.1f}%)")
                print(f"    F2 Score: {row['f2_score']:.3f} ({f2_change:+.1f}%)")
                print(f"    Accuracy: {row['accuracy']:.3f} ({acc_change:+.1f}%)")
            
            # Find best model for F2 score
            best_f2_idx = metrics_df['f2_score'].idxmax()
            if pd.notna(best_f2_idx):
                best_model = metrics_df.loc[best_f2_idx]
                
                print("\nBEST MODEL FOR F2 SCORE:")
                print(f"  Model: {best_f2_idx}")
                print(f"  Type: {best_model['display_name']}")
                print(f"  F2 Score: {best_model['f2_score']:.3f}")
                print(f"  TP: {best_model['true_positives']:.1f}")
                print(f"  FP: {best_model['false_positives']:.1f}")
                print(f"  FN: {best_model['false_negatives']:.1f}")
                print(f"  TN: {best_model['true_negatives']:.1f}")
            
            # Analysis of tradeoffs
            print("\nTRADEOFF ANALYSIS:")
            
            # 1. Find model with highest TP
            best_tp_idx = metrics_df['true_positives'].idxmax()
            if pd.notna(best_tp_idx):
                model_tp = metrics_df.loc[best_tp_idx]
                print(f"  Best TP: {model_tp['display_name']} ({best_tp_idx}) with {model_tp['true_positives']:.1f} true positives")
            
            # 2. Find model with lowest FN (critical for F2)
            best_fn_idx = metrics_df['false_negatives'].idxmin()
            if pd.notna(best_fn_idx):
                model_fn = metrics_df.loc[best_fn_idx]
                print(f"  Lowest FN: {model_fn['display_name']} ({best_fn_idx}) with {model_fn['false_negatives']:.1f} false negatives")
            
            # Check for TP/FP tradeoff (a key concern for F2)
            print("\n  TP/FP TRADEOFF ANALYSIS:")
            tp_fp_ratio = metrics_df['true_positives'] / metrics_df['false_positives'].replace(0, np.nan)
            best_ratio_idx = tp_fp_ratio.idxmax()
            if pd.notna(best_ratio_idx):
                model_ratio = metrics_df.loc[best_ratio_idx]
                print(f"  Best TP/FP ratio: {model_ratio['display_name']} ({best_ratio_idx}) with ratio {tp_fp_ratio[best_ratio_idx]:.2f}")
                print(f"    TP: {model_ratio['true_positives']:.1f}, FP: {model_ratio['false_positives']:.1f}")
            
            # Final recommendations
            print("\nRECOMMENDATIONS BASED ON F2 OPTIMIZATION PRIORITY:")
            
            llm_models = metrics_df[metrics_df['model_type'] != 'sentence_transformer_tf_idf']
            if not llm_models.empty:
                # Check if LLMs outperform sentence transformers on key metrics
                st_avg_f2 = st_metrics['f2_score'].values[0]
                llm_avg_f2 = llm_models['f2_score'].mean()
                
                if llm_avg_f2 > st_avg_f2:
                    print("  1. LLM models generally outperform Sentence Transformer/TF-IDF on F2 score")
                    
                    # Find best LLM model for F2
                    best_llm_f2_idx = llm_models['f2_score'].idxmax()
                    if pd.notna(best_llm_f2_idx):
                        best_llm = llm_models.loc[best_llm_f2_idx]
                        print(f"  2. Recommended model: {best_llm_f2_idx} (F2: {best_llm['f2_score']:.3f})")
                        
                        # Analyze why this model performs well
                        print(f"  3. This model performs well because:", end=" ")
                        
                        reasons = []
                        if best_llm['true_positives'] > st_metrics['true_positives'].values[0]:
                            reasons.append(f"higher TP ({best_llm['true_positives']:.1f} vs {st_metrics['true_positives'].values[0]:.1f})")
                        
                        if best_llm['false_negatives'] < st_metrics['false_negatives'].values[0]:
                            reasons.append(f"lower FN ({best_llm['false_negatives']:.1f} vs {st_metrics['false_negatives'].values[0]:.1f})")
                        
                        if reasons:
                            print(", ".join(reasons))
                        else:
                            print("balanced performance across metrics")
                else:
                    print("  1. Sentence Transformer/TF-IDF models still perform better on F2 score compared to LLMs")
                    print(f"  2. Recommended model: Best sentence transformer (F2: {st_avg_f2:.3f})")
            
            # Suggestions for improvement
            print("\n  SUGGESTIONS FOR IMPROVEMENT:")
            # Look at where the best model is weak
            if pd.notna(best_f2_idx):
                best_model = metrics_df.loc[best_f2_idx]
                
                weaknesses = []
                if best_model['false_negatives'] > metrics_df['false_negatives'].min():
                    weakness_diff = best_model['false_negatives'] - metrics_df['false_negatives'].min()
                    weaknesses.append(f"reduce false negatives (current: {best_model['false_negatives']:.1f}, potential improvement: {weakness_diff:.1f})")
                
                if best_model['true_positives'] < metrics_df['true_positives'].max():
                    weakness_diff = metrics_df['true_positives'].max() - best_model['true_positives']
                    weaknesses.append(f"increase true positives (current: {best_model['true_positives']:.1f}, potential improvement: {weakness_diff:.1f})")
                
                if weaknesses:
                    print("    Areas for further optimization: " + ", ".join(weaknesses))
                else:
                    print("    The best model already optimizes the key metrics for F2 score")
        else:
            print("\nNo Sentence Transformer/TF-IDF baseline for comparison")

# Run the analysis on confusion matrix data
analyze_confusion_matrix_improvements(confusion_matrix_df)

In [ ]:
# Cell [10] - Project-Based LLM Type Improvement Whisker Charts
# Purpose: Generate detailed LLM improvement analysis with whisker charts by project and model type
# Dependencies: pandas, matplotlib, seaborn, numpy
# Breadcrumbs: Visualization -> LLM Improvement Analysis

def create_project_llm_improvement_whiskers(confusion_df):
    """
    Create comprehensive charts showing LLM type improvements over Sentence Transformer baselines
    
    Parameters:
        confusion_df (pd.DataFrame): DataFrame containing confusion matrix metrics with columns:
                                    - project_name, model_type, model_name, metric, value
                                    
    Returns:
        None: Displays visualizations and summary tables directly
        
    Notes:
        - Creates one chart per project with grouped whiskers for each LLM type
        - Shows percentage improvements for TP, FP, FN (inverted), TN relative to baseline
        - Uses Sentence Transformer/TF-IDF as baseline for comparison calculations
        - Generates detailed summary statistics tables by LLM type and metric
        - Provides model instance breakdown showing individual performance
        - Uses consistent color scheme across all visualizations
        - Includes mean value annotations on boxplots for easy interpretation
    """
    if confusion_df.empty:
        print("No data available for visualization")
        return
    
    # Metric labels for display
    metric_labels = {
        'true_positives': 'TP',
        'false_positives': 'FP',
        'false_negatives': 'FN (inv)',
        'true_negatives': 'TN'
    }
    
    # Get list of all projects
    projects = confusion_df['project_name'].unique()
    
    # Process each project separately
    for project in projects:
        print(f"\nAnalyzing project: {project}")
        
        # Get data for this project
        project_data = confusion_df[confusion_df['project_name'] == project]
        
        # Extract sentence transformer data for baseline
        st_data = project_data[project_data['model_type'] == 'sentence_transformer_tf_idf']
        
        if st_data.empty:
            print(f"  No Sentence Transformer baseline found for project {project}, skipping")
            continue
            
        # Calculate baseline values for each metric
        st_baseline = {}
        for metric in metric_labels.keys():
            metric_data = st_data[st_data['metric'] == metric]
            if not metric_data.empty:
                st_baseline[metric] = metric_data['value'].mean()
                print(f"  {metric} baseline: {st_baseline[metric]:.1f}")
            else:
                st_baseline[metric] = None
                print(f"  {metric} baseline: None")
        
        # Get all LLM data for this project
        llm_data = project_data[project_data['model_type'] != 'sentence_transformer_tf_idf']
        
        if llm_data.empty:
            print(f"  No LLM models found for project {project}, skipping")
            continue
        
        # We need to correctly identify the LLM model types
        # First, check what we have
        print("\nLLM model information available:")
        display(llm_data[['model_type', 'model_name']].drop_duplicates().head(10))
        
        # The actual LLM types (like "claude-3-7-sonnet") are in model_type
        # Let's use that as our grouping variable instead of model_name
        llm_types = llm_data['model_type'].unique()
        print(f"  Found {len(llm_types)} LLM types: {', '.join(llm_types)}")
        
        # Create a DataFrame to hold improvement data
        improvement_data = []
        
        # Calculate improvements for each LLM type and metric
        for llm_type in llm_types:
            type_data = llm_data[llm_data['model_type'] == llm_type]
            
            for metric in metric_labels.keys():
                metric_data = type_data[type_data['metric'] == metric]
                
                if not metric_data.empty and st_baseline[metric] is not None and st_baseline[metric] > 0:
                    for _, row in metric_data.iterrows():
                        # Calculate percentage improvement
                        improvement = ((row['value'] - st_baseline[metric]) / st_baseline[metric]) * 100
                        
                        # For FN, invert so negative means fewer FNs (which is good)
                        if metric == 'false_negatives':
                            improvement = -improvement
                            
                        improvement_data.append({
                            'LLM Type': llm_type,
                            'Model Name': row['model_name'],
                            'Metric': metric,
                            'Metric Display': metric_labels[metric],
                            'Improvement (%)': improvement,
                            'Original Value': row['value'],
                            'Baseline Value': st_baseline[metric]
                        })
        
        # Skip if no improvement data
        if not improvement_data:
            print(f"  No valid improvement data for project {project}, skipping")
            continue
            
        # Convert to DataFrame
        improvements_df = pd.DataFrame(improvement_data)
        
        # Create plot
        plt.figure(figsize=(16, 7))
        
        # Create grouped box plot
        # X-axis grouped by LLM type, with each group having 4 boxes (TP, FP, FN, TN)
        ax = sns.boxplot(
            x='LLM Type',
            y='Improvement (%)',
            hue='Metric Display',
            data=improvements_df,
            palette='Set3',
            width=0.8
        )
        
        # Add reference line at 0%
        ax.axhline(y=0, color='black', linestyle='-', alpha=0.5)
        
        # Add mean value labels to each column
        # First, calculate means for each combination
        means = improvements_df.groupby(['LLM Type', 'Metric Display'])['Improvement (%)'].mean().reset_index()
        
        # Get the x-tick positions
        x_ticks = list(range(len(llm_types)))
        
        # Iterate through means and add text annotations
        for llm_idx, llm_type in enumerate(llm_types):
            metrics_for_llm = means[means['LLM Type'] == llm_type]
            
            # Calculate width of each box
            box_width = 0.8 / len(metric_labels)
            
            for metric_idx, (_, row) in enumerate(metrics_for_llm.iterrows()):
                metric = row['Metric Display']
                mean_val = row['Improvement (%)']
                
                # Calculate position (adjust as needed based on box positions)
                x_pos = llm_idx + (metric_idx - 1.5) * box_width
                
                # Add text annotation
                ax.text(
                    x_pos, 
                    mean_val + (5 if mean_val > 0 else -15),  # Adjust y position
                    f"{mean_val:.1f}%",
                    ha='center',
                    va='bottom' if mean_val > 0 else 'top',
                    fontsize=9,
                    fontweight='bold',
                    color='blue'
                )
        
        # Customize the plot
        plt.title(f"LLM Type Improvements Over Sentence Transformer - Project: {project}", fontsize=16)
        plt.ylabel("Percentage Improvement (%)", fontsize=14)
        plt.xlabel("LLM Type", fontsize=14)
        
        # Adjust legend
        plt.legend(title='Metric', fontsize=12)
        
        # Add grid
        plt.grid(axis='y', linestyle='--', alpha=0.3)
        
        # Adjust layout
        plt.tight_layout()
        plt.xticks(rotation=45, ha='right')  # Rotate model names for readability
        plt.show()
        
        # Create a summary table
        print("\nSummary Statistics by LLM Type and Metric:")
        
        # Pivot the data for a better summary
        summary = pd.pivot_table(
            improvements_df, 
            values='Improvement (%)',
            index=['LLM Type'], 
            columns=['Metric Display'],
            aggfunc='mean'
        )
        
        # Add average column
        summary['Average'] = summary.mean(axis=1)
        
        # Sort by average improvement
        summary = summary.sort_values('Average', ascending=False)
        
        # Display the summary
        display(summary)
        
        # Show detailed model instance breakdown
        print("\nDetailed breakdown by model instance:")
        model_breakdown = pd.pivot_table(
            improvements_df,
            values='Improvement (%)',
            index=['LLM Type', 'Model Name'],
            columns=['Metric Display'],
            aggfunc='mean'
        )
        
        # Add average column
        model_breakdown['Average'] = model_breakdown.mean(axis=1)
        
        # Sort by LLM Type and then by Average
        model_breakdown = model_breakdown.sort_values(['LLM Type', 'Average'], ascending=[True, False])
        
        display(model_breakdown)

# Run the visualization function
create_project_llm_improvement_whiskers(confusion_matrix_df)

In [ ]:
# Cell [11] - Role-Based MCC Comparison by LLM Type and Project
# Purpose: Comprehensive analysis of Matthews Correlation Coefficient across roles and model types
# Dependencies: pandas, matplotlib, seaborn, numpy, re
# Breadcrumbs: Visualization -> Role-Based Performance Analysis

def create_role_mcc_comparison(model_df):
    """
    Create comprehensive visualizations comparing MCC scores across different roles by LLM type per project
    
    Parameters:
        model_df (pd.DataFrame): DataFrame containing model performance data from Neo4j with columns:
                                - project_name, model_type, model_data_parsed, display_model_type
                                
    Returns:
        None: Displays visualizations and summary tables directly
        
    Notes:
        - Extracts role information from model names (actor, judge, meta_judge)
        - Focuses on Matthews Correlation Coefficient as primary metric
        - Creates grouped boxplots comparing roles within each model type
        - Maintains specific role order: actor, judge, meta_judge
        - Handles final_score as meta_judge role to avoid duplicates
        - Generates separate visualizations for model types and individual LLMs
        - Includes statistical summary tables with mean and count aggregations
        - Uses stripplot overlay for individual data point visibility
    """
    if model_df.empty:
        print("No data available for visualization")
        return
    
    # Extract role information from model names
    role_data = []
    
    # Process model data to extract roles and MCC scores
    for _, row in model_df.iterrows():
        project_name = row['project_name']
        model_type = row['display_model_type']
        model_data = row['model_data_parsed']
        
        if not model_data or not isinstance(model_data, dict):
            continue
            
        # Find Matthews correlation data
        mcc_data = None
        for metric_key, models in model_data.items():
            if 'matthews' in metric_key.lower() or 'matthews_corr' in metric_key.lower():
                mcc_data = models
                break
                
        if not mcc_data:
            continue
        
        # Find the final_score (meta_judge) if it exists
        final_score = None
        for model_name, score in mcc_data.items():
            if 'final_score' in model_name.lower():
                final_score = score
                break
            
        # Extract role from model names and organize data
        for model_name, score in mcc_data.items():
            # More precise role extraction
            if 'actor' in model_name.lower() and 'meta' not in model_name.lower():
                role = 'actor'
            elif 'judge' in model_name.lower() and 'meta' not in model_name.lower():
                role = 'judge'
            elif 'meta_judge' in model_name.lower() or 'meta judge' in model_name.lower() or 'final_score' in model_name.lower():
                # Skip all meta_judge entries except final_score to avoid duplicates
                if 'final_score' not in model_name.lower():
                    continue
                role = 'meta_judge'
            else:
                # Skip unknown roles
                continue
                
            # Extract LLM name (if available)
            llm_match = re.search(r'(claude|gpt|gemini|llama|mistral)', model_name.lower())
            llm_name = llm_match.group(1) if llm_match else "unknown"
            
            # Add to dataset
            role_data.append({
                'project': project_name,
                'model_type': model_type,
                'model_name': model_name,
                'role': role,
                'llm_name': llm_name,
                'mcc_score': score
            })
            
        # If we found a final_score but it wasn't added through the loop above
        # (because it wasn't named properly), add it explicitly
        if final_score is not None:
            meta_judge_exists = any(item['role'] == 'meta_judge' and item['model_type'] == model_type 
                                   for item in role_data if item['project'] == project_name)
            
            if not meta_judge_exists:
                # Add final_score as meta_judge
                role_data.append({
                    'project': project_name,
                    'model_type': model_type,
                    'model_name': 'final_score',
                    'role': 'meta_judge',
                    'llm_name': "unknown",
                    'mcc_score': final_score
                })
    
    # Convert to DataFrame
    role_df = pd.DataFrame(role_data)
    
    if role_df.empty:
        print("No role-based MCC data found in the dataset")
        return
    
    # Filter out Sentence Transformer/TF-IDF data points
    role_df = role_df[role_df['model_type'] != 'Sentence Transformer/TF-IDF']
        
    # Display summary of extracted data
    print(f"Extracted MCC data for {role_df['project'].nunique()} projects, "
          f"{role_df['model_type'].nunique()} model types, and "
          f"{role_df['role'].nunique()} roles")
    
    # Show unique roles found
    print("\nRoles found in the data:")
    for role in sorted(role_df['role'].unique()):
        print(f"  - {role}")
    
    # Set the order of roles (actor, judge, meta_judge)
    role_order = ['actor', 'judge', 'meta_judge']
    
    # Create visualizations for each project
    for project in role_df['project'].unique():
        project_data = role_df[role_df['project'] == project]
        
        print(f"\nCreating visualization for project: {project}")
        print(f"  Model types: {', '.join(project_data['model_type'].unique())}")
        print(f"  Roles: {', '.join(project_data['role'].unique())}")
        
        # Create figure
        plt.figure(figsize=(14, 8))
        
        # Create grouped boxplot with specific role order
        ax = sns.boxplot(
            x='model_type',
            y='mcc_score',
            hue='role',
            data=project_data,
            palette='Set2',
            width=0.8,
            showfliers=False,
            hue_order=role_order  # Specify the order of roles
        )
        
        # Add individual data points with jitter
        sns.stripplot(
            x='model_type',
            y='mcc_score',
            hue='role',
            data=project_data,
            palette='Set2',
            size=6,
            alpha=0.7,
            jitter=True,
            dodge=True,
            marker='o',
            linewidth=1,
            edgecolor='black',
            legend=False,
            hue_order=role_order  # Specify the order of roles
        )
        
        # Customize the plot
        plt.title(f"Matthews Correlation Coefficient by Role and Model Type - {project}", fontsize=16)
        plt.xlabel("Model Type", fontsize=14)
        plt.ylabel("Matthews Correlation Coefficient", fontsize=14)
        plt.ylim(0, 1.0)  # MCC is typically in range [-1, 1] but most values are positive
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        
        # Update the legend with the correct order
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles=handles[:3], labels=labels[:3], title="Role", fontsize=12)
        
        # Set x-axis tick labels at 45 degrees with right alignment
        plt.xticks(rotation=45, ha='right')
        
        # Add mean value annotations
        means = project_data.groupby(['model_type', 'role'])['mcc_score'].mean().reset_index()
        
        for _, row in means.iterrows():
            # Calculate position based on the ordered roles
            model_types = sorted(project_data['model_type'].unique())
            model_idx = model_types.index(row['model_type'])
            
            # Get the role index based on our ordered list
            if row['role'] in role_order:
                role_idx = role_order.index(row['role'])
                
                # Calculate offset for each role
                offset = (role_idx - (len(role_order) - 1) / 2) * (0.8 / len(role_order))
                
                # Add annotation
                plt.text(
                    model_idx + offset, 
                    row['mcc_score'] + 0.03,
                    f"{row['mcc_score']:.3f}",
                    ha='center',
                    va='bottom',
                    fontsize=9,
                    fontweight='bold'
                )
        
        plt.tight_layout()
        plt.savefig(f"{project}_role_mcc_comparison.png", dpi=300, bbox_inches='tight')
        plt.show()
        
        # Create a summary table with roles in the correct order
        print("\nSummary Statistics by Model Type and Role:")
        
        # Ensure roles appear in the desired order in the pivot table
        summary = pd.pivot_table(
            project_data,
            values='mcc_score',
            index=['model_type'],
            columns=['role'],
            aggfunc=['mean', 'count']
        )
        
        # Flatten hierarchical index for better display
        summary.columns = [f"{agg}_{role}" for agg, role in summary.columns]
        display(summary)
        
        # Optional: LLM-specific analysis if LLM info available
        if 'llm_name' in project_data.columns and project_data['llm_name'].nunique() > 1:
            print("\nLLM-specific MCC Comparison:")
            
            # Filter out unknown LLMs
            llm_data = project_data[project_data['llm_name'] != 'unknown']
            
            if not llm_data.empty:
                # Create figure for LLM comparison
                plt.figure(figsize=(16, 8))
                
                # Create grouped boxplot by LLM with specified role order
                sns.boxplot(
                    x='llm_name',
                    y='mcc_score',
                    hue='role',
                    data=llm_data,
                    palette='Set3',
                    width=0.8,
                    showfliers=False,
                    hue_order=role_order  # Specify role order
                )
                
                # Add jittered points
                sns.stripplot(
                    x='llm_name',
                    y='mcc_score',
                    hue='role',
                    data=llm_data,
                    palette='Set3',
                    size=6,
                    alpha=0.7,
                    jitter=True,
                    dodge=True,
                    marker='o',
                    linewidth=1,
                    edgecolor='black',
                    legend=False,
                    hue_order=role_order  # Specify role order
                )
                
                # Customize
                plt.title(f"Matthews Correlation by Role and LLM - {project}", fontsize=16)
                plt.xlabel("LLM Name", fontsize=14)
                plt.ylabel("Matthews Correlation Coefficient", fontsize=14)
                plt.ylim(0, 1.0)
                plt.grid(axis='y', linestyle='--', alpha=0.7)
                
                # Update the legend with the correct order
                handles, labels = plt.gca().get_legend_handles_labels()
                plt.legend(handles=handles[:3], labels=labels[:3], title="Role", fontsize=12)
                
                # Set x-axis tick labels at 45 degrees with right alignment
                plt.xticks(rotation=45, ha='right')
                
                plt.tight_layout()
                plt.savefig(f"{project}_llm_role_mcc_comparison.png", dpi=300, bbox_inches='tight')
                plt.show()
                
                # Show LLM summary table
                llm_summary = pd.pivot_table(
                    llm_data,
                    values='mcc_score',
                    index=['llm_name'],
                    columns=['role'],
                    aggfunc=['mean', 'count']
                )
                
                llm_summary.columns = [f"{agg}_{role}" for agg, role in llm_summary.columns]
                display(llm_summary)

# Run the role-based MCC comparison
create_role_mcc_comparison(model_df)